# GEE python

GEE 也有python接口，可以和command shell，web application和jupyter交互。和JS相比，python API可以在脚本间分享代码，允许序列调用GEE，有很多绘图选项，比如matplotlib，seaborn，plotly，bokeh，pygal，ggplot和altair等，可以常用python库一起使用。最直接的调用GEE的方式可以查看GEE原文档 [Python installation](https://developers.google.com/earth-engine/python_install)，这里主要参考[GEEMAP](https://github.com/giswqs/geemap)，记录如何使用吴秋生老师开发的geemap，教程在这里：[mdpi-gee.pdf](https://spatial.utk.edu/ppt/mdpi-gee.pdf)，还有[bilibili geemap](https://space.bilibili.com/527404442/channel/detail?cid=132674)，以此简单记录下关于GEE python的使用。

## geemap Installation

geemap 可以看成是gee python接口的高级封装，所以自然要先安装gee python接口所需要的东西。

有两种方式可以快速上手，一种是在云端运行，一种是本地。云端运行需要配置colab，这部分可以参考 [Earth Engine Python API & Colab](https://docs.google.com/presentation/d/e/2PACX-1vTFAWIEBg3aVhY5dImnTzTDmbgnu9A4pEOW6OUdpoCLvxD1ot3UI0qbcD90oXsR0CJdWQz-kAUE_2tb/pub?start=false&loop=false&delayms=3000)。然后在云端配置GEE环境，并验证。colab每次重新加载进来之后都需要重新配置环境。所以目前个人觉得不是特别方便，可能配合使用google云服务比较好，这里还是以本地为主了。

最基础的GEE python接口需要的环境一般有：

- python
- 包管理器（conda/pip）
- GEE Python API
- Python数据科学计算库

这些在本repo的environment.yml文件中都已经包含了，也已经包含geemap了，所以在运行本repo前的安装中都已经安装过了，即在项目根目录下打开终端，执行以下语句即可：

```Shell
conda env create -f environment.yml
```

安装过了就不必再执行了。

In [1]:
import ee

导入ee没有报错，说明已经安装了gee的python接口了。

然后需要获取credentials才可以使用API，这是一个授权本地代表谷歌账号访问GEE的凭证。

原始方法是在终端使用下面命令：

``` Shell
earthengine authenticate
```

按照提示操作，会得到一个激活码，复制这个码，然后切回终端，粘贴这个激活码。点击回车，等待验证即可。不过这里因为使用了geemap，所以直接执行下面的语句，会调用授权过程相关的代码，能起到同样的作用，如下所示。

因为我不是第一次授权，所以下面直接就跳出来GEE的地图界面了。如果你是第一次授权可以参考吴秋生老师的原视频：https://www.bilibili.com/video/BV1fi4y147rL

In [2]:
import geemap

In [4]:
m = geemap.Map()
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

上面的地图是可交互的，可以试着在地图范围内滚动鼠标轮，就能实现地图缩放。总之，这样就安装好了 gee python接口和 geemap。

## geemap 基本使用介绍

开始geemap之前，值得一提的是原gee 本地命令是有命令行命令的，这个在官方文档中有详细的介绍，简单说一下常用的几个命令，剩余的在使用过程中如果又需要可以随时查看相关官方文档的介绍。官方文档地址：https://link.zhihu.com/?target=https%3A//developers.google.com/earth-engine/command_line  

命令概览可以通过下面的命令行代码实现：

In [5]:
!earthengine --help

usage: earthengine [-h] [--ee_config EE_CONFIG]
                   [--service_account_file SERVICE_ACCOUNT_FILE]
                   [--project PROJECT_OVERRIDE]
                   {authenticate,acl,asset,cp,create,ls,licenses,du,mv,model,rm,set_project,task,unset_project,upload,upload_manifest,upload_table_manifest}
                   ...

Earth Engine Command Line Interface.

optional arguments:
  -h, --help            show this help message and exit
  --ee_config EE_CONFIG
                        Path to the earthengine configuration file. Defaults
                        to "~/.config\earthengine\credentials".
  --service_account_file SERVICE_ACCOUNT_FILE
                        Path to a service account credentialsfile. Overrides
                        any ee_config if specified.
  --project PROJECT_OVERRIDE
                        Specifies a Google Cloud Platform Project id to
                        override the call.

Commands:
  {authenticate,acl,asset,cp,create,ls,licenses,d

几个常用工具：

- authenticate:认证本地账户。
- acl: 
    - get 查询资源的信息
    - set 将资源设置为公共或者私有
- asset: 
    - info 查看资源的属性
    - set 为资源设置特殊属性字段
- cp: 复制一份新的资源
- mv: 重命名资源

下面进入正题。

## 底图展示

首先是底图的展示。geemap集合了许多basemap，这里就不一一记录了，直接展示默认的GEE底图

In [6]:
import geemap

Map = geemap.Map(center=[40, -100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## 导入图层

可以尝试导入不同的数据，可以使用上面的交互地图左上角的数据搜索功能，如下图所示

![](pictures/QQ截图20210122110809.png)

也可以和JS的用法一样，直接导入，如下代码：

In [7]:
import ee
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
states = ee.FeatureCollection("TIGER/2018/States")

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Eninge layers to Map
Map.addLayer(dem, vis_params, 'SRTM DEM', True, 0.5)
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(landsat7, {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200}, 'Landsat 7')
Map.addLayer(states, {}, "US States")

找到图层栏，如下图所示：

![](pictures/QQ截图20210122111050.png)

可以随意切换底图。

这样基本的功能就了解了，接下来就边根据例子边介绍GEE的使用方式。